# Building a Retrieval Augmented Generation (RAG) Chatbot

In [2]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
!pip install -qU langchain-mistralai
!pip install -qU langchain-chroma

# To correct UnstructuredXMLLoader errors
# !pip install "unstructured>=0.6.7"

## Setup

In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for Langsmith: ")

Enter API key for Langsmith:  ········


## Components

In [6]:
import getpass
import os

# Chat model

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

Enter API key for Mistral AI:  ········


In [7]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings

# Chat model
llm = ChatMistralAI(model="mistral-large-latest")

# Embeddings model
embeddings = MistralAIEmbeddings(model="mistral-embed")

C:\Users\Laura GALINDO\anaconda3\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [8]:
from langchain_chroma import Chroma

# Vector store
vector_store = Chroma(embedding_function=embeddings)

## Loading JSON and XML files

Transforming the data JSON and XML files we got online into LangChain Document objects

#### Test: loading a single JSON file

In [11]:
import json
from pathlib import Path
from pprint import pprint

file_path="../parcoursup/parcoursup_data.json"
data = json.loads(Path(file_path).read_text())
pprint(data)

[{'acceptations': 0,
  'annee_du_bac': 2021,
  'doublette': 'Art,Art',
  'formation': 'Licence Chimie',
  'propositions_d_admissions': 0.0,
  'voeux': 0},
 {'acceptations': 0,
  'annee_du_bac': 2021,
  'doublette': 'Art,Art',
  'formation': 'Licence Langues et littÃ©ratures franÃ§aises',
  'propositions_d_admissions': 0.0,
  'voeux': 0},
 {'acceptations': 0,
  'annee_du_bac': 2021,
  'doublette': 'Art,Art',
  'formation': 'Licence Philosophie, EpistÃ©mologie',
  'propositions_d_admissions': 0.0,
  'voeux': 0},
 {'acceptations': 0,
  'annee_du_bac': 2021,
  'doublette': 'Art,Art',
  'formation': 'Licence Physique',
  'propositions_d_admissions': 0.0,
  'voeux': 0},
 {'acceptations': 0,
  'annee_du_bac': 2021,
  'doublette': 'Art,Art',
  'formation': 'Licence Pluri Droit - Sciences Ã©co - AES',
  'propositions_d_admissions': 0.0,
  'voeux': 0},
 {'acceptations': 0,
  'annee_du_bac': 2021,
  'doublette': 'Art,Art',
  'formation': 'Licence Pluri Langues',
  'propositions_d_admissions': 0.0

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="../parcoursup/parcoursup_data.json",
    jq_schema=".[]",
    text_content=False
)

documents = loader.load()
pprint(documents[0])

Document(metadata={'source': 'C:\\Users\\Laura GALINDO\\Documents\\Academique_et_pro\\Epitech Digital School\\Bootcamp Chatbot IA\\chatbot-orientation-pro\\parcoursup\\parcoursup_data.json', 'seq_num': 1}, page_content='{"annee_du_bac": 2021, "doublette": "Art,Art", "formation": "Licence Chimie", "voeux": 0, "propositions_d_admissions": 0, "acceptations": 0}')


#### Test: loading a single XML file

In [14]:
from bs4 import BeautifulSoup

with open('../onisep/Onisep_Fiches_Formations.xml', 'r', encoding="utf-8") as f:
    data = f.read()

bs_data = BeautifulSoup(data, 'xml')

print(bs_data.prettify()[:500])

<?xml version="1.0" encoding="utf-8"?>
<formations xmlns:exsl="http://exslt.org/common">
 <formation>
  <identifiant>
   FOR.1000
  </identifiant>
  <code_scolarite>
   16X20003
  </code_scolarite>
  <libelle_complet>
   mastère spé. management de la qualité, de la sécurité et de l'environnement
  </libelle_complet>
  <sigle/>
  <libelle_generique/>
  <libelle_specifique/>
  <type_option/>
  <type_Formation>
   <type_formation_sigle>
    mastère spé.
   </type_formation_sigle>
   <type_formation


In [15]:
from langchain_community.document_loaders import UnstructuredXMLLoader
from pprint import pprint

loader = UnstructuredXMLLoader('../onisep/Onisep_Fiches_Formations.xml')

documents = loader.load()

print(documents[0].page_content[:500])

FOR.1000

16X20003

mastère spé. management de la qualité, de la sécurité et de l'environnement

mastère spé.

MASTERE SPE

mastère spécialisé

"1 an"

https://www.francecompetences.fr/recherche/rncp/37084/

200

Spécialités pluritechnologiques de la production

R

Contrôle, prévention, entretien

niveau 7

REF.423

bac + 6

16X

Label de la Conférence des grandes écoles (CGE)

Formation inscrite au RNCP

MET.878

chargé/e hygiène sécurité environnement (HSE)

chargée hygiène sécurité environnem


#### Loading multiple JSON and XML documents into LangChain Document objects

In [17]:
import os
import json
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import JSONLoader, UnstructuredXMLLoader

# To correct UnstructuredXMLLoader errors
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def load_json_documents(file_path: str) -> list:
    """
    Load a JSON file using JSONLoader and return a list of Documents
    """
    loader = JSONLoader(
        file_path=file_path,
        jq_schema=".[]",
        text_content=False,
    )
    try:
        documents = loader.load()
        print(f"[JSON] Loaded file: {file_path} ({len(documents)} document(s))")
        return documents
    except Exception as e:
        print(f"Error while loading JSON file {file_path} : {e}")
        return []

def load_xml_documents(file_path: str) -> list:
    """
    Load an XML file using UnstructuredXMLLoader and return a list of Documents
    """
    loader = UnstructuredXMLLoader(file_path)
    try:
        documents = loader.load()
        print(f"[XML] Loaded file: {file_path} ({len(documents)} document(s))")
        return documents
    except Exception as e:
        print(f"Error while loading XML file {file_path} : {e}")
        return []

def load_documents(file_paths: list) -> list:
    """
    For each file in the list, we determine its extension and call the appropriate loader
    (JSONLoader or UnstructuredXMLLoader), and then return a complete list of Documents
    """
    documents = []
    for fp in file_paths:
        ext = Path(fp).suffix.lower()
        if ext == ".json":
            documents.extend(load_json_documents(fp))
        elif ext == ".xml":
            documents.extend(load_xml_documents(fp))
        else:
            print(f"Unsupported file (extension {ext}) : {fp}")
    return documents

def export_documents(documents: list):
    """
    Save the Document objects to a text file for inspection or debugging
    """
    with open("loaded_documents.txt", "w", encoding="utf-8") as f:
        for doc in documents:
            f.write(f"Source: {doc.metadata.get('source')}\n")
            f.write(f"Content:\n{doc.page_content}\n\n---\n\n")
    
    print("Documents saved to 'loaded_documents.txt'")

In [18]:
# List of JSON and XML files to load
file_list = [
    "../parcoursup/parcoursup_data.json",
    "../mon_master/monmaster_data.json",
    "../onisep/Onisep_Fiches_Formations.xml",
    "../onisep/Onisep_Fiches_Metiers.xml"
]

# Load all the documents
all_documents = load_documents(file_list)

print(f"\nTotal number of loaded documents: {len(all_documents)}")

# export_documents(all_documents)

[JSON] Loaded file: ../parcoursup/parcoursup_data.json (12610 document(s))
[JSON] Loaded file: ../mon_master/monmaster_data.json (19603 document(s))
[XML] Loaded file: ../onisep/Onisep_Fiches_Formations.xml (1 document(s))
[XML] Loaded file: ../onisep/Onisep_Fiches_Metiers.xml (1 document(s))

Total number of loaded documents: 32215


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def dynamic_split_documents(documents, base_chunk_size=1000, min_chunks=3, max_chunks=200):
    """
    Dynamically split a list of documents into smaller chunks based on document size
    
    Parameters:
        - documents: List of LangChain Document objects
        - base_chunk_size: Default chunk size in characters
        - min_chunks: Minimum number of chunks per document
        - max_chunks: Maximum number of chunks per document
    
    Returns:
        - List of split documents
    """
    all_splits = []
    
    for doc in documents:
        doc_length = len(doc.page_content)
        
        # Dynamically adjust the chunk size:
        # - Ensure chunks are not too small by using at least `base_chunk_size`
        # - Ensure chunks are not too big by keeping at least `min_chunks` per document
        # - This balances the number and size of chunks for better retrieval
        chunk_size = max(base_chunk_size, doc_length // max_chunks)  # Prevents chunks from being too tiny
        chunk_size = min(chunk_size, doc_length // min_chunks)  # Ensures we don't have too few large chunks
        
        # Define the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_size // 5,  # 20% overlap
            add_start_index=True # Keeps track of where in the original document each chunk came from
        )
        
        # Split the document and extend the list
        splits = text_splitter.split_documents([doc])
        # print(f"Document: {doc.metadata.get('source')} | Original Length: {len(doc.page_content)} | Chunks Created: {len(splits)}")
        
        all_splits.extend(splits) # Adds all the elements in splits to all_splits, one by one
    
    print(f"\nTotal number of chunks created: {len(all_splits)}")
    return all_splits

def export_split_documents(chunks):
    """
    Save the split documents to a text file with delimiters between chunks

    Each document chunk's source and content are written to 'split_documents.txt', 
    with '---' as a separator between different chunks.

    Parameters:
        - chunks (list): A list of LangChain Document objects after splitting

    Output:
        - File containing all split document chunks with delimiters
    """
    with open("split_documents.txt", "w", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            # Add delimiter before every chunk except the first one
            if i > 0:
                f.write("\n---\n\n")

            f.write(f"Source: {chunk.metadata.get('source')}\n")
            f.write(f"Content:\n{chunk.page_content}\n")

    print("Split documents saved to 'split_documents.txt'")

In [20]:
# Split the loaded documents
split_documents = dynamic_split_documents(all_documents)

# export_split_documents(split_documents)


Total number of chunks created: 129801


## Storing Document splits

Indexing our chunks of documents to be able to vector search over them at runtime, which entails embedding the contents of each chunk of Document into the Chroma vector store

In [ ]:
import time

batch_size = 10  # Process 10 chunks of documents at a time

for i in range(0, len(split_documents), batch_size):
    batch = split_documents[i:i + batch_size]
    
    try:
        document_ids = vector_store.add_documents(documents=batch)
        print(f"Batch {i//batch_size + 1}: {document_ids[:3]}")
    except Exception as e:
        print(f"Error in batch {i//batch_size + 1}: {e}")
    
    time.sleep(2)  # Wait 2 seconds before the next batch

Batch 1: ['e4e290e7-3ad0-4d26-8c13-078ae83824f4', '719fe31e-7cbd-4152-9cf2-5262553ffa01', '0493492c-a637-4bcb-a4f7-2f597daca379']
Batch 2: ['e5f77537-85ef-4f34-aff0-363cacea1bde', 'fbe2271e-0e23-4ea2-ba17-b123030ba502', '08b201ae-99f7-4e6e-816d-39bdafb8973f']
Batch 3: ['895ed64f-8598-4ead-bc3c-1ca658ce5249', '47841687-c351-4b7f-9d73-0525cdf5fa7c', '4eb28f8d-692d-45e6-9ce5-1923424e1996']
Batch 4: ['672fd908-9ff9-45b8-abb6-c32d9dda1ea1', 'c52f9663-bab3-46a7-9813-9f0070d94350', '68411707-5536-4bac-83b3-191e0b1f4f83']
Batch 5: ['5ed49deb-2601-4586-9940-04aaa6b365af', 'fbe995aa-7adc-40e3-a586-c35b649757d3', 'e30d0280-519d-4bd8-aa43-db34cefb7319']
Batch 6: ['8d23730b-b8e0-4c0f-b819-1aa2ee9b0cae', '5490c4cc-12ca-4817-b27b-96eee03a7f57', '82aee1a9-d916-4aae-bb87-a0e29e12043a']
Batch 7: ['2c0b0c90-66bc-4a4d-bb3b-deee8947624e', '301e314f-e85c-4d97-9274-86a07f7eb0cd', 'c29c8d6d-975f-4380-90b1-91c9603d25ee']
Batch 8: ['3b836467-4f2d-424b-b03d-d65ec5b3a343', '78cad6ef-f8ea-4dd9-b7c4-764e09ca0eb8', 

## Creating the RAG chain with LangChain

Creating a chain that, based on a user question, will retrieve relevant Documents from Chroma and provide them in context to the MistralAI language model to generate an answer

In [ ]:
# Interface that interrogates the vector store
retriever = vecteur_store.as_retriever(search_kwargs={"k": 5})  # the k most relevant chunks of documents

